In [1]:
import numpy as np
from copy import deepcopy
import random
from typing import Any, Tuple, Optional, Sequence

In [2]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.tensorboard import SummaryWriter

/home/hhchung/dyngraph-uda/.venv/lib/python3.8/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
from torch_geometric.data import Data
from torch_geometric.utils import to_undirected
from torch_geometric.nn import SAGEConv
from torch_geometric.loader import DataLoader
from torch.autograd import Function
from pytorch_adapt.validators import IMValidator

In [4]:
from ogb.nodeproppred import PygNodePropPredDataset
from ogb.nodeproppred.evaluate import Evaluator

In [5]:
import sys
sys.path.append("..")
from dataset import temp_partition_arxiv

In [6]:
dataset_name = 'ogbn-arxiv'
dataset = PygNodePropPredDataset(name = dataset_name, root='/home/hhchung/data/ogb-data')

In [7]:
random_seed = 42
torch.manual_seed(random_seed)
torch.cuda.manual_seed(random_seed)
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False
np.random.seed(random_seed)
random.seed(random_seed)

## Model Structure ##

In [8]:
class TwoLayerGraphSAGE(nn.Module):
    def __init__(self, in_dim, hidden_dim, out_dim, dropout=0.2):
        super().__init__()
        self.dropout = dropout
        self.conv1 = SAGEConv(in_dim, hidden_dim)
        self.conv2 = SAGEConv(hidden_dim, out_dim)
        
    def forward(self, x, edge_index):
        x = self.conv1(x, edge_index)
        x = F.elu(x)
        x = F.dropout(x, p=self.dropout)
        
        x = self.conv2(x, edge_index)
        x = F.elu(x)
        return x
    

class ThreeLayerGraphSAGE(nn.Module):
    def __init__(self, in_dim, hidden_dim, out_dim, dropout=0.2):
        super().__init__()
        self.dropout = dropout
        self.conv1 = SAGEConv(in_dim, hidden_dim)
        self.conv2 = SAGEConv(hidden_dim, hidden_dim)
        self.conv3 = SAGEConv(hidden_dim, out_dim)
        
    def forward(self, x, edge_index):
        x = self.conv1(x, edge_index)
        x = F.elu(x)
        x = F.dropout(x, p=self.dropout)
        
        x = self.conv2(x, edge_index)
        x = F.elu(x)
        x = F.dropout(x, p=self.dropout)
        
        x = self.conv3(x, edge_index)
        return x
    

    
class MLPHead(nn.Module):
    def __init__(self, in_dim, hidden_dim, out_dim, dropout=0.2):
        super().__init__()
        self.dropout = dropout
        self.linear1 = nn.Linear(in_dim, hidden_dim)
        self.linear2 = nn.Linear(hidden_dim, out_dim)
    
    def forward(self, x):
        x1 = self.linear1(x)
        x1 = F.elu(x1)
        x1 = F.dropout(x1, p=self.dropout)
        output = self.linear2(x1)
        # x = F.softmax(x, dim=1)
        # features = x1
        return output
    
class DomainClassifier(nn.Module):
    def __init__(self, in_dim, hidden_dim, out_dim, dropout=0.2):
        super().__init__()
        self.dropout = dropout
        self.linear1 = nn.Linear(in_dim, hidden_dim)
        self.linear2 = nn.Linear(hidden_dim, out_dim)

    def forward(self, x):
        x = self.linear1(x)
        x = F.elu(x)
        x = F.dropout(x, p=self.dropout)
        x = self.linear2(x)
        return x
    
    
class ReverseLayerF(Function):

    @staticmethod
    def forward(ctx, x, alpha):
        ctx.alpha = alpha

        return x.view_as(x)

    @staticmethod
    def backward(ctx, grad_output):
        output = grad_output.neg() * ctx.alpha

        return output, None

In [9]:
def train(encoder, mlp, optimizer, data):
    encoder.train()
    mlp.train()
    
    out = mlp(encoder(data.x, data.edge_index))
    out = F.log_softmax(out, dim=1)
    loss = F.nll_loss(out[data.train_mask], data.y[data.train_mask])
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()
    return loss.item()
    
@torch.no_grad()
def test(encoder, mlp, data, evaluator):
    encoder.eval()
    mlp.eval()
    
    out = mlp(encoder(data.x, data.edge_index))
    out = F.log_softmax(out, dim=1)
    val_loss = F.nll_loss(out[data.val_mask], data.y[data.val_mask]).item()
    y_pred = out.argmax(dim=-1, keepdim=True)
    val_acc = evaluator.eval({
        'y_true': data.y[data.val_mask].unsqueeze(1),
        'y_pred': y_pred[data.val_mask],
    })['acc']
    
    return val_loss, val_acc

In [10]:
def adapt(encoder, classifier, domain_classifier, src_data, tgt_data, optimizer, e, epochs, lambda_coeff=1, device="cpu"):
    encoder.train()
    classifier.train()
    domain_classifier.train()
    
    p = e / epochs
    alpha = (2. / (1. + np.exp(-10 * p)) - 1) * lambda_coeff
    
    src_size = src_data.x.shape[0]
    src_dom_label = torch.zeros(src_size).long().to(device)
    src_f = encoder(src_data.x, src_data.edge_index)
    src_cls_output = classifier(src_f)
    src_dom_output = domain_classifier(ReverseLayerF.apply(src_f, alpha))
    loss_src_label = F.nll_loss(F.log_softmax(src_cls_output, dim=1)[src_data.train_mask], src_data.y[src_data.train_mask])
    loss_src_domain = F.nll_loss(F.log_softmax(src_dom_output, dim=1)[src_data.train_mask], src_dom_label[src_data.train_mask])
    
    tgt_size = tgt_data.x.shape[0]
    tgt_dom_label = torch.ones(tgt_size).long().to(device)
    tgt_f = encoder(tgt_data.x, tgt_data.edge_index)
    tgt_dom_output = domain_classifier(ReverseLayerF.apply(tgt_f, alpha))
    loss_tgt_domain = F.nll_loss(F.log_softmax(tgt_dom_output, dim=1)[tgt_data.train_mask], tgt_dom_label[tgt_data.train_mask])
    
    total_size = src_size + tgt_size
    loss = loss_src_label + loss_src_domain + loss_tgt_domain
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

    return loss.item(), loss_src_label.item(), loss_src_domain.item(), loss_tgt_domain.item()
    
@torch.no_grad()
def adapt_test(encoder, classifier, domain_classifier, src_data, tgt_data, e, epochs, lambda_coeff=1, device="cpu"):
    encoder.eval()
    classifier.eval()
    domain_classifier.eval()
    
    p = e / epochs
    alpha = (2. / (1. + np.exp(-10 * p)) - 1) * lambda_coeff
    
    src_size = src_data.x.shape[0]
    src_dom_label = torch.zeros(src_size).long().to(device)
    src_f = encoder(src_data.x, src_data.edge_index)
    src_cls_output = classifier(src_f)
    src_dom_output = domain_classifier(ReverseLayerF.apply(src_f, alpha))
    loss_src_label = F.nll_loss(F.log_softmax(src_cls_output, dim=1)[src_data.val_mask], src_data.y[src_data.val_mask])
    loss_src_domain = F.nll_loss(F.log_softmax(src_dom_output, dim=1)[src_data.val_mask], src_dom_label[src_data.val_mask])
    
    tgt_size = tgt_data.x.shape[0]
    tgt_dom_label = torch.ones(tgt_size).long().to(device)
    tgt_f = encoder(tgt_data.x, tgt_data.edge_index)
    tgt_dom_output = domain_classifier(ReverseLayerF.apply(tgt_f, alpha))
    loss_tgt_domain = F.nll_loss(F.log_softmax(tgt_dom_output, dim=1)[tgt_data.val_mask], tgt_dom_label[tgt_data.val_mask])
    
    total_size = src_size + tgt_size
    loss = loss_src_label + loss_src_domain + loss_tgt_domain
    
    tgt_logits = classifier(tgt_f)
    
    return loss.item(), loss_src_label.item(), loss_src_domain.item(), loss_tgt_domain.item(), tgt_logits

## Load Data ##

In [11]:
dataset = PygNodePropPredDataset(name = dataset_name, root='/home/hhchung/data/ogb-data')
data = dataset[0]
data.edge_index = to_undirected(data.edge_index, data.num_nodes) # mimicking barlow twins repo

## Data Partition ##

* Train: 0-2011
* Val: 2012
* Test:
** 2013-2014 (then adapt 2012-2013 adapt-val 2014)
** 2015-2016 (then adapt 2014-2015 adapt-val 2016)
** 2017-2018 (then adapt 2016-2017 adapt-val 2018)
** 2019-2020

## Source Training Stage ##

* Train: 0-2011
* Val: 2012

In [12]:
feat_dim = data.x.shape[1]
class_dim = data.y
hidden_dim = 128
emb_dim = 256
encoder = TwoLayerGraphSAGE(feat_dim, hidden_dim, emb_dim)
mlp = MLPHead(emb_dim, emb_dim // 4, 40)

optimizer = torch.optim.Adam(list(encoder.parameters()) + list(mlp.parameters()), lr=1e-3)
epochs = 500

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
encoder = encoder.to(device)
mlp = mlp.to(device)

In [13]:
data_2012_2013 = temp_partition_arxiv(data, year_bound=[-1,2012,2013], proportion=1.0)
data_2012_2013 = data_2012_2013.to(device)

In [14]:
best_acc = 0
best_encoder = None
best_mlp = None
evaluator = Evaluator(name='ogbn-arxiv')
for e in range(1, epochs + 1):
    train_loss = train(encoder, mlp, optimizer, data_2012_2013)
    val_loss, val_acc = test(encoder, mlp, data_2012_2013, evaluator)
    print(f"Epoch:{e}/{epochs} Train Loss:{round(train_loss,4)} Val Loss:{round(val_loss,4)} Val Acc:{round(val_acc, 4)}")
    if val_acc > best_acc:
        best_acc = val_acc
        best_encoder = deepcopy(encoder)
        best_mlp = deepcopy(mlp)

encoder = deepcopy(best_encoder)
mlp = deepcopy(best_mlp)

Epoch:1/500 Train Loss:3.7146 Val Loss:3.6349 Val Acc:0.1565
Epoch:2/500 Train Loss:3.6308 Val Loss:3.5643 Val Acc:0.1977
Epoch:3/500 Train Loss:3.5491 Val Loss:3.4867 Val Acc:0.2036
Epoch:4/500 Train Loss:3.4588 Val Loss:3.4016 Val Acc:0.2045
Epoch:5/500 Train Loss:3.3554 Val Loss:3.3091 Val Acc:0.2045
Epoch:6/500 Train Loss:3.2474 Val Loss:3.2373 Val Acc:0.2045
Epoch:7/500 Train Loss:3.159 Val Loss:3.2137 Val Acc:0.2045
Epoch:8/500 Train Loss:3.1197 Val Loss:3.2375 Val Acc:0.2045
Epoch:9/500 Train Loss:3.1278 Val Loss:3.2435 Val Acc:0.2045
Epoch:10/500 Train Loss:3.1227 Val Loss:3.1977 Val Acc:0.2045
Epoch:11/500 Train Loss:3.0835 Val Loss:3.1572 Val Acc:0.2034
Epoch:12/500 Train Loss:3.0444 Val Loss:3.1318 Val Acc:0.2026
Epoch:13/500 Train Loss:3.014 Val Loss:3.1054 Val Acc:0.1984
Epoch:14/500 Train Loss:3.0002 Val Loss:3.1003 Val Acc:0.1997
Epoch:15/500 Train Loss:2.985 Val Loss:3.0831 Val Acc:0.2009
Epoch:16/500 Train Loss:2.9754 Val Loss:3.0729 Val Acc:0.2068
Epoch:17/500 Train L

Epoch:137/500 Train Loss:1.5349 Val Loss:1.7459 Val Acc:0.5122
Epoch:138/500 Train Loss:1.5291 Val Loss:1.7466 Val Acc:0.5043
Epoch:139/500 Train Loss:1.5236 Val Loss:1.7382 Val Acc:0.5161
Epoch:140/500 Train Loss:1.5244 Val Loss:1.7433 Val Acc:0.5106
Epoch:141/500 Train Loss:1.5167 Val Loss:1.7368 Val Acc:0.5099
Epoch:142/500 Train Loss:1.5141 Val Loss:1.739 Val Acc:0.5114
Epoch:143/500 Train Loss:1.5089 Val Loss:1.7335 Val Acc:0.511
Epoch:144/500 Train Loss:1.5028 Val Loss:1.7282 Val Acc:0.5117
Epoch:145/500 Train Loss:1.5014 Val Loss:1.7215 Val Acc:0.5094
Epoch:146/500 Train Loss:1.4982 Val Loss:1.7283 Val Acc:0.5172
Epoch:147/500 Train Loss:1.5045 Val Loss:1.7207 Val Acc:0.5159
Epoch:148/500 Train Loss:1.4934 Val Loss:1.7122 Val Acc:0.5207
Epoch:149/500 Train Loss:1.4897 Val Loss:1.7171 Val Acc:0.5167
Epoch:150/500 Train Loss:1.4878 Val Loss:1.7082 Val Acc:0.5201
Epoch:151/500 Train Loss:1.4852 Val Loss:1.7078 Val Acc:0.5161
Epoch:152/500 Train Loss:1.4839 Val Loss:1.7085 Val Acc:0

Epoch:279/500 Train Loss:1.2351 Val Loss:1.5383 Val Acc:0.5649
Epoch:280/500 Train Loss:1.236 Val Loss:1.5319 Val Acc:0.5652
Epoch:281/500 Train Loss:1.2323 Val Loss:1.5374 Val Acc:0.5649
Epoch:282/500 Train Loss:1.2312 Val Loss:1.5346 Val Acc:0.5672
Epoch:283/500 Train Loss:1.2324 Val Loss:1.5378 Val Acc:0.5697
Epoch:284/500 Train Loss:1.2283 Val Loss:1.5345 Val Acc:0.5644
Epoch:285/500 Train Loss:1.2241 Val Loss:1.5422 Val Acc:0.5692
Epoch:286/500 Train Loss:1.2288 Val Loss:1.5324 Val Acc:0.5719
Epoch:287/500 Train Loss:1.2237 Val Loss:1.5325 Val Acc:0.5716
Epoch:288/500 Train Loss:1.2263 Val Loss:1.5329 Val Acc:0.5709
Epoch:289/500 Train Loss:1.2195 Val Loss:1.537 Val Acc:0.572
Epoch:290/500 Train Loss:1.2249 Val Loss:1.5207 Val Acc:0.574
Epoch:291/500 Train Loss:1.2193 Val Loss:1.5284 Val Acc:0.5717
Epoch:292/500 Train Loss:1.219 Val Loss:1.547 Val Acc:0.5695
Epoch:293/500 Train Loss:1.2172 Val Loss:1.5275 Val Acc:0.5697
Epoch:294/500 Train Loss:1.2225 Val Loss:1.5292 Val Acc:0.568

Epoch:421/500 Train Loss:1.1007 Val Loss:1.502 Val Acc:0.5818
Epoch:422/500 Train Loss:1.1004 Val Loss:1.5084 Val Acc:0.5773
Epoch:423/500 Train Loss:1.1003 Val Loss:1.4965 Val Acc:0.5812
Epoch:424/500 Train Loss:1.0951 Val Loss:1.4919 Val Acc:0.5851
Epoch:425/500 Train Loss:1.0918 Val Loss:1.5123 Val Acc:0.5754
Epoch:426/500 Train Loss:1.098 Val Loss:1.4917 Val Acc:0.584
Epoch:427/500 Train Loss:1.0969 Val Loss:1.495 Val Acc:0.5824
Epoch:428/500 Train Loss:1.0977 Val Loss:1.5085 Val Acc:0.582
Epoch:429/500 Train Loss:1.0968 Val Loss:1.5057 Val Acc:0.581
Epoch:430/500 Train Loss:1.0992 Val Loss:1.5102 Val Acc:0.5823
Epoch:431/500 Train Loss:1.0925 Val Loss:1.5002 Val Acc:0.5848
Epoch:432/500 Train Loss:1.0905 Val Loss:1.5101 Val Acc:0.5765
Epoch:433/500 Train Loss:1.0896 Val Loss:1.5148 Val Acc:0.5773
Epoch:434/500 Train Loss:1.0907 Val Loss:1.5079 Val Acc:0.5812
Epoch:435/500 Train Loss:1.0899 Val Loss:1.504 Val Acc:0.5812
Epoch:436/500 Train Loss:1.0882 Val Loss:1.5054 Val Acc:0.5837

In [15]:
best_acc

0.5866355866355867

# Prequential Evaluation at Subsequent Time Steps #

In [16]:
def continual_adapt(src_data, tgt_split, encoder, mlp, device, lambda_coeff=1, lr=1e-3):
    print("Start partitioning data...")
    tgt_data = temp_partition_arxiv(data, year_bound=tgt_split, proportion=1.0)
    tgt_data.to(device)
    print("Finish partitioning data...")
    
    tgt_encoder, tgt_mlp = deepcopy(encoder), deepcopy(mlp)
    domain_classifier = DomainClassifier(emb_dim, emb_dim // 4, 2).to(device)
    tgt_optimizer = torch.optim.Adam(list(tgt_encoder.parameters()) + list(tgt_mlp.parameters()) + list(domain_classifier.parameters()), lr=lr)
    
    epochs = 500
    validator = IMValidator()
    best_val_score = -np.inf
    best_tgt_encoder, best_tgt_mlp = None, None
    best_val_loss_src_label, best_val_loss_src_domain, best_val_loss_tgt_domain = None, None, None
    for e in range(1, epochs + 1):
        train_loss, train_loss_src_label, train_loss_src_domain, train_loss_tgt_domain = adapt(tgt_encoder, tgt_mlp, domain_classifier, src_data, tgt_data, tgt_optimizer, e, epochs, lambda_coeff, device=device)
        val_loss, val_loss_src_label, val_loss_src_domain, val_loss_tgt_domain, val_tgt_logits = adapt_test(tgt_encoder, tgt_mlp, domain_classifier, src_data, tgt_data, e, epochs, lambda_coeff, device=device)
        # print(val_tgt_logits)
        val_score = validator(target_train={'logits': val_tgt_logits})
        if val_score > best_val_score:
            best_val_score = val_score
            best_tgt_encoder, best_tgt_mlp = deepcopy(tgt_encoder), deepcopy(tgt_mlp)
            best_val_loss_src_label = val_loss_src_label
            best_val_loss_src_domain = val_loss_src_domain
            best_val_loss_tgt_domain = val_loss_tgt_domain
            
        print(f'Epoch {e}/{epochs} Train Src Label Loss: {round(train_loss_src_label,3)} Train Src Domain Loss: {round(train_loss_src_domain,3)} Train Tgt Domain Loss: {round(train_loss_tgt_domain,3)}')
        print(f'Epoch {e}/{epochs} Val Src Label Loss: {round(val_loss_src_label,3)} Val Src Domain Loss: {round(val_loss_src_domain,3)} Val Tgt Domain Loss: {round(val_loss_tgt_domain,3)} Val Score {round(val_score,3)}')


    tgt_encoder = deepcopy(best_tgt_encoder)
    tgt_mlp = deepcopy(best_tgt_mlp)
    
    return tgt_encoder, tgt_mlp, best_val_loss_src_label, best_val_loss_src_domain, best_val_loss_tgt_domain

In [17]:
lambda_coeff = 0.05
lr = 1e-3
test_acc_list = []

## 2013-2014 ##

* Test:
** 2013-2014 (then adapt 2012-2013 adapt-val 2014)

In [18]:
data_2013_2015 = temp_partition_arxiv(data, year_bound=[-1,2013,2015], proportion=1.0)
data_2013_2015 = data_2013_2015.to(device)

In [19]:
test_loss, test_acc = test(encoder, mlp, data_2013_2015, evaluator)
test_acc_list.append(test_acc)

In [20]:
print(f"Test Loss: {round(test_loss,3)} Test Acc: {round(test_acc,3)}")

Test Loss: 1.476 Test Acc: 0.591


In [21]:
encoder_2012_2014_2015, mlp_2012_2014_2015, val_loss_src_label, val_loss_src_domain, val_loss_tgt_domain = continual_adapt(data_2012_2013, [2012,2014,2015], encoder, mlp, device, lambda_coeff=lambda_coeff, lr=lr)

Start partitioning data...
Finish partitioning data...
Epoch 1/500 Train Src Label Loss: 1.055 Train Src Domain Loss: 0.774 Train Tgt Domain Loss: 0.651
Epoch 1/500 Val Src Label Loss: 1.557 Val Src Domain Loss: 0.696 Val Tgt Domain Loss: 0.703 Val Score 1.898
Epoch 2/500 Train Src Label Loss: 1.115 Train Src Domain Loss: 0.693 Train Tgt Domain Loss: 0.7
Epoch 2/500 Val Src Label Loss: 1.515 Val Src Domain Loss: 0.692 Val Tgt Domain Loss: 0.707 Val Score 1.875
Epoch 3/500 Train Src Label Loss: 1.056 Train Src Domain Loss: 0.676 Train Tgt Domain Loss: 0.703
Epoch 3/500 Val Src Label Loss: 1.521 Val Src Domain Loss: 0.736 Val Tgt Domain Loss: 0.677 Val Score 1.862
Epoch 4/500 Train Src Label Loss: 1.064 Train Src Domain Loss: 0.703 Train Tgt Domain Loss: 0.667
Epoch 4/500 Val Src Label Loss: 1.535 Val Src Domain Loss: 0.77 Val Tgt Domain Loss: 0.655 Val Score 1.852
Epoch 5/500 Train Src Label Loss: 1.09 Train Src Domain Loss: 0.722 Train Tgt Domain Loss: 0.645
Epoch 5/500 Val Src Label L

Epoch 47/500 Train Src Label Loss: 1.026 Train Src Domain Loss: 0.663 Train Tgt Domain Loss: 0.673
Epoch 47/500 Val Src Label Loss: 1.511 Val Src Domain Loss: 0.742 Val Tgt Domain Loss: 0.679 Val Score 1.913
Epoch 48/500 Train Src Label Loss: 1.019 Train Src Domain Loss: 0.666 Train Tgt Domain Loss: 0.67
Epoch 48/500 Val Src Label Loss: 1.512 Val Src Domain Loss: 0.743 Val Tgt Domain Loss: 0.677 Val Score 1.913
Epoch 49/500 Train Src Label Loss: 1.021 Train Src Domain Loss: 0.669 Train Tgt Domain Loss: 0.67
Epoch 49/500 Val Src Label Loss: 1.495 Val Src Domain Loss: 0.744 Val Tgt Domain Loss: 0.678 Val Score 1.905
Epoch 50/500 Train Src Label Loss: 1.019 Train Src Domain Loss: 0.667 Train Tgt Domain Loss: 0.67
Epoch 50/500 Val Src Label Loss: 1.515 Val Src Domain Loss: 0.744 Val Tgt Domain Loss: 0.678 Val Score 1.9
Epoch 51/500 Train Src Label Loss: 1.011 Train Src Domain Loss: 0.666 Train Tgt Domain Loss: 0.668
Epoch 51/500 Val Src Label Loss: 1.498 Val Src Domain Loss: 0.747 Val Tgt 

Epoch 89/500 Train Src Label Loss: 0.988 Train Src Domain Loss: 0.66 Train Tgt Domain Loss: 0.662
Epoch 89/500 Val Src Label Loss: 1.512 Val Src Domain Loss: 0.76 Val Tgt Domain Loss: 0.671 Val Score 1.939
Epoch 90/500 Train Src Label Loss: 0.994 Train Src Domain Loss: 0.663 Train Tgt Domain Loss: 0.658
Epoch 90/500 Val Src Label Loss: 1.496 Val Src Domain Loss: 0.757 Val Tgt Domain Loss: 0.672 Val Score 1.939
Epoch 91/500 Train Src Label Loss: 0.989 Train Src Domain Loss: 0.66 Train Tgt Domain Loss: 0.662
Epoch 91/500 Val Src Label Loss: 1.508 Val Src Domain Loss: 0.754 Val Tgt Domain Loss: 0.675 Val Score 1.943
Epoch 92/500 Train Src Label Loss: 0.989 Train Src Domain Loss: 0.657 Train Tgt Domain Loss: 0.665
Epoch 92/500 Val Src Label Loss: 1.503 Val Src Domain Loss: 0.751 Val Tgt Domain Loss: 0.676 Val Score 1.945
Epoch 93/500 Train Src Label Loss: 0.988 Train Src Domain Loss: 0.653 Train Tgt Domain Loss: 0.667
Epoch 93/500 Val Src Label Loss: 1.501 Val Src Domain Loss: 0.757 Val Tg

Epoch 132/500 Train Src Label Loss: 0.953 Train Src Domain Loss: 0.651 Train Tgt Domain Loss: 0.664
Epoch 132/500 Val Src Label Loss: 1.532 Val Src Domain Loss: 0.758 Val Tgt Domain Loss: 0.675 Val Score 1.97
Epoch 133/500 Train Src Label Loss: 0.959 Train Src Domain Loss: 0.653 Train Tgt Domain Loss: 0.664
Epoch 133/500 Val Src Label Loss: 1.519 Val Src Domain Loss: 0.766 Val Tgt Domain Loss: 0.667 Val Score 1.971
Epoch 134/500 Train Src Label Loss: 0.954 Train Src Domain Loss: 0.657 Train Tgt Domain Loss: 0.658
Epoch 134/500 Val Src Label Loss: 1.522 Val Src Domain Loss: 0.768 Val Tgt Domain Loss: 0.666 Val Score 1.969
Epoch 135/500 Train Src Label Loss: 0.953 Train Src Domain Loss: 0.659 Train Tgt Domain Loss: 0.654
Epoch 135/500 Val Src Label Loss: 1.524 Val Src Domain Loss: 0.763 Val Tgt Domain Loss: 0.673 Val Score 1.967
Epoch 136/500 Train Src Label Loss: 0.952 Train Src Domain Loss: 0.653 Train Tgt Domain Loss: 0.66
Epoch 136/500 Val Src Label Loss: 1.527 Val Src Domain Loss: 0

Epoch 176/500 Train Src Label Loss: 0.924 Train Src Domain Loss: 0.648 Train Tgt Domain Loss: 0.66
Epoch 176/500 Val Src Label Loss: 1.534 Val Src Domain Loss: 0.773 Val Tgt Domain Loss: 0.666 Val Score 2.001
Epoch 177/500 Train Src Label Loss: 0.918 Train Src Domain Loss: 0.657 Train Tgt Domain Loss: 0.65
Epoch 177/500 Val Src Label Loss: 1.525 Val Src Domain Loss: 0.777 Val Tgt Domain Loss: 0.667 Val Score 2.002
Epoch 178/500 Train Src Label Loss: 0.921 Train Src Domain Loss: 0.663 Train Tgt Domain Loss: 0.647
Epoch 178/500 Val Src Label Loss: 1.528 Val Src Domain Loss: 0.77 Val Tgt Domain Loss: 0.671 Val Score 1.999
Epoch 179/500 Train Src Label Loss: 0.918 Train Src Domain Loss: 0.653 Train Tgt Domain Loss: 0.656
Epoch 179/500 Val Src Label Loss: 1.528 Val Src Domain Loss: 0.764 Val Tgt Domain Loss: 0.677 Val Score 2.003
Epoch 180/500 Train Src Label Loss: 0.918 Train Src Domain Loss: 0.645 Train Tgt Domain Loss: 0.662
Epoch 180/500 Val Src Label Loss: 1.54 Val Src Domain Loss: 0.7

...

Epoch 305/500 Train Src Label Loss: 0.812 Train Src Domain Loss: 0.665 Train Tgt Domain Loss: 0.628
Epoch 305/500 Val Src Label Loss: 1.597 Val Src Domain Loss: 0.789 Val Tgt Domain Loss: 0.661 Val Score 2.1
Epoch 306/500 Train Src Label Loss: 0.812 Train Src Domain Loss: 0.653 Train Tgt Domain Loss: 0.64
Epoch 306/500 Val Src Label Loss: 1.582 Val Src Domain Loss: 0.77 Val Tgt Domain Loss: 0.681 Val Score 2.092
Epoch 307/500 Train Src Label Loss: 0.812 Train Src Domain Loss: 0.634 Train Tgt Domain Loss: 0.658
Epoch 307/500 Val Src Label Loss: 1.591 Val Src Domain Loss: 0.76 Val Tgt Domain Loss: 0.687 Val Score 2.086
Epoch 308/500 Train Src Label Loss: 0.821 Train Src Domain Loss: 0.626 Train Tgt Domain Loss: 0.669
Epoch 308/500 Val Src Label Loss: 1.598 Val Src Domain Loss: 0.779 Val Tgt Domain Loss: 0.671 Val Score 2.094
Epoch 309/500 Train Src Label Loss: 0.814 Train Src Domain Loss: 0.644 Train Tgt Domain Loss: 0.651
Epoch 309/500 Val Src Label Loss: 1.58 Val Src Domain Loss: 0.799

Epoch 348/500 Train Src Label Loss: 0.781 Train Src Domain Loss: 0.638 Train Tgt Domain Loss: 0.648
Epoch 348/500 Val Src Label Loss: 1.63 Val Src Domain Loss: 0.794 Val Tgt Domain Loss: 0.661 Val Score 2.12
Epoch 349/500 Train Src Label Loss: 0.773 Train Src Domain Loss: 0.646 Train Tgt Domain Loss: 0.642
Epoch 349/500 Val Src Label Loss: 1.632 Val Src Domain Loss: 0.791 Val Tgt Domain Loss: 0.663 Val Score 2.126
Epoch 350/500 Train Src Label Loss: 0.782 Train Src Domain Loss: 0.645 Train Tgt Domain Loss: 0.643
Epoch 350/500 Val Src Label Loss: 1.618 Val Src Domain Loss: 0.79 Val Tgt Domain Loss: 0.667 Val Score 2.134
Epoch 351/500 Train Src Label Loss: 0.78 Train Src Domain Loss: 0.64 Train Tgt Domain Loss: 0.646
Epoch 351/500 Val Src Label Loss: 1.62 Val Src Domain Loss: 0.783 Val Tgt Domain Loss: 0.672 Val Score 2.135
Epoch 352/500 Train Src Label Loss: 0.772 Train Src Domain Loss: 0.638 Train Tgt Domain Loss: 0.648
Epoch 352/500 Val Src Label Loss: 1.611 Val Src Domain Loss: 0.785

Epoch 394/500 Train Src Label Loss: 0.744 Train Src Domain Loss: 0.639 Train Tgt Domain Loss: 0.638
Epoch 394/500 Val Src Label Loss: 1.648 Val Src Domain Loss: 0.799 Val Tgt Domain Loss: 0.653 Val Score 2.168
Epoch 395/500 Train Src Label Loss: 0.742 Train Src Domain Loss: 0.646 Train Tgt Domain Loss: 0.633
Epoch 395/500 Val Src Label Loss: 1.653 Val Src Domain Loss: 0.787 Val Tgt Domain Loss: 0.665 Val Score 2.161
Epoch 396/500 Train Src Label Loss: 0.741 Train Src Domain Loss: 0.639 Train Tgt Domain Loss: 0.644
Epoch 396/500 Val Src Label Loss: 1.637 Val Src Domain Loss: 0.785 Val Tgt Domain Loss: 0.672 Val Score 2.152
Epoch 397/500 Train Src Label Loss: 0.738 Train Src Domain Loss: 0.625 Train Tgt Domain Loss: 0.653
Epoch 397/500 Val Src Label Loss: 1.658 Val Src Domain Loss: 0.786 Val Tgt Domain Loss: 0.67 Val Score 2.151
Epoch 398/500 Train Src Label Loss: 0.729 Train Src Domain Loss: 0.633 Train Tgt Domain Loss: 0.647
Epoch 398/500 Val Src Label Loss: 1.648 Val Src Domain Loss: 

Epoch 438/500 Train Src Label Loss: 0.7 Train Src Domain Loss: 0.644 Train Tgt Domain Loss: 0.627
Epoch 438/500 Val Src Label Loss: 1.681 Val Src Domain Loss: 0.809 Val Tgt Domain Loss: 0.651 Val Score 2.189
Epoch 439/500 Train Src Label Loss: 0.701 Train Src Domain Loss: 0.65 Train Tgt Domain Loss: 0.629
Epoch 439/500 Val Src Label Loss: 1.678 Val Src Domain Loss: 0.801 Val Tgt Domain Loss: 0.663 Val Score 2.192
Epoch 440/500 Train Src Label Loss: 0.701 Train Src Domain Loss: 0.638 Train Tgt Domain Loss: 0.637
Epoch 440/500 Val Src Label Loss: 1.693 Val Src Domain Loss: 0.789 Val Tgt Domain Loss: 0.67 Val Score 2.19
Epoch 441/500 Train Src Label Loss: 0.704 Train Src Domain Loss: 0.631 Train Tgt Domain Loss: 0.645
Epoch 441/500 Val Src Label Loss: 1.696 Val Src Domain Loss: 0.783 Val Tgt Domain Loss: 0.67 Val Score 2.192
Epoch 442/500 Train Src Label Loss: 0.706 Train Src Domain Loss: 0.622 Train Tgt Domain Loss: 0.649
Epoch 442/500 Val Src Label Loss: 1.705 Val Src Domain Loss: 0.794

Epoch 481/500 Train Src Label Loss: 0.67 Train Src Domain Loss: 0.639 Train Tgt Domain Loss: 0.63
Epoch 481/500 Val Src Label Loss: 1.738 Val Src Domain Loss: 0.806 Val Tgt Domain Loss: 0.659 Val Score 2.221
Epoch 482/500 Train Src Label Loss: 0.671 Train Src Domain Loss: 0.633 Train Tgt Domain Loss: 0.633
Epoch 482/500 Val Src Label Loss: 1.71 Val Src Domain Loss: 0.797 Val Tgt Domain Loss: 0.67 Val Score 2.222
Epoch 483/500 Train Src Label Loss: 0.669 Train Src Domain Loss: 0.624 Train Tgt Domain Loss: 0.642
Epoch 483/500 Val Src Label Loss: 1.733 Val Src Domain Loss: 0.797 Val Tgt Domain Loss: 0.669 Val Score 2.229
Epoch 484/500 Train Src Label Loss: 0.672 Train Src Domain Loss: 0.631 Train Tgt Domain Loss: 0.642
Epoch 484/500 Val Src Label Loss: 1.736 Val Src Domain Loss: 0.813 Val Tgt Domain Loss: 0.656 Val Score 2.227
Epoch 485/500 Train Src Label Loss: 0.661 Train Src Domain Loss: 0.638 Train Tgt Domain Loss: 0.631
Epoch 485/500 Val Src Label Loss: 1.728 Val Src Domain Loss: 0.8

In [22]:
val_loss_src_label, val_loss_src_domain, val_loss_tgt_domain

(1.7197505235671997, 0.8173037171363831, 0.6473246812820435)

## 2015-2016 ##

* Test:
** 2015-2016 (then adapt 2014-2015 adapt-val 2016)

In [23]:
data_2015_2017 = temp_partition_arxiv(data, year_bound=[-1,2015,2017], proportion=1.0)
data_2015_2017 = data_2015_2017.to(device)

In [24]:
test_loss, test_acc = test(encoder_2012_2014_2015, mlp_2012_2014_2015, data_2015_2017, evaluator)
test_acc_list.append(test_acc)

In [25]:
print(f"Test Loss: {round(test_loss,3)} Test Acc: {round(test_acc,3)}")

Test Loss: 1.891 Test Acc: 0.543


In [26]:
encoder_2014_2016_2017, mlp_2014_2016_2017, val_loss_src_label, val_loss_src_domain, val_loss_tgt_domain = continual_adapt(data_2012_2013, [2014,2016,2017], encoder_2012_2014_2015, mlp_2012_2014_2015, device, lambda_coeff=lambda_coeff, lr=lr)

Start partitioning data...
Finish partitioning data...
Epoch 1/500 Train Src Label Loss: 0.661 Train Src Domain Loss: 0.739 Train Tgt Domain Loss: 0.679
Epoch 1/500 Val Src Label Loss: 1.828 Val Src Domain Loss: 0.696 Val Tgt Domain Loss: 0.658 Val Score 2.178
Epoch 2/500 Train Src Label Loss: 0.742 Train Src Domain Loss: 0.677 Train Tgt Domain Loss: 0.695
Epoch 2/500 Val Src Label Loss: 1.742 Val Src Domain Loss: 0.772 Val Tgt Domain Loss: 0.568 Val Score 2.217
Epoch 3/500 Train Src Label Loss: 0.666 Train Src Domain Loss: 0.724 Train Tgt Domain Loss: 0.618
Epoch 3/500 Val Src Label Loss: 1.752 Val Src Domain Loss: 0.823 Val Tgt Domain Loss: 0.526 Val Score 2.221
Epoch 4/500 Train Src Label Loss: 0.689 Train Src Domain Loss: 0.752 Train Tgt Domain Loss: 0.587
Epoch 4/500 Val Src Label Loss: 1.716 Val Src Domain Loss: 0.799 Val Tgt Domain Loss: 0.552 Val Score 2.226
Epoch 5/500 Train Src Label Loss: 0.696 Train Src Domain Loss: 0.709 Train Tgt Domain Loss: 0.624
Epoch 5/500 Val Src Lab

Epoch 48/500 Train Src Label Loss: 0.642 Train Src Domain Loss: 0.635 Train Tgt Domain Loss: 0.629
Epoch 48/500 Val Src Label Loss: 1.75 Val Src Domain Loss: 0.7 Val Tgt Domain Loss: 0.555 Val Score 2.238
Epoch 49/500 Train Src Label Loss: 0.639 Train Src Domain Loss: 0.627 Train Tgt Domain Loss: 0.637
Epoch 49/500 Val Src Label Loss: 1.766 Val Src Domain Loss: 0.692 Val Tgt Domain Loss: 0.561 Val Score 2.238
Epoch 50/500 Train Src Label Loss: 0.634 Train Src Domain Loss: 0.62 Train Tgt Domain Loss: 0.644
Epoch 50/500 Val Src Label Loss: 1.767 Val Src Domain Loss: 0.688 Val Tgt Domain Loss: 0.563 Val Score 2.24
Epoch 51/500 Train Src Label Loss: 0.639 Train Src Domain Loss: 0.615 Train Tgt Domain Loss: 0.647
Epoch 51/500 Val Src Label Loss: 1.747 Val Src Domain Loss: 0.689 Val Tgt Domain Loss: 0.563 Val Score 2.246
Epoch 52/500 Train Src Label Loss: 0.637 Train Src Domain Loss: 0.614 Train Tgt Domain Loss: 0.646
Epoch 52/500 Val Src Label Loss: 1.754 Val Src Domain Loss: 0.699 Val Tgt 

Epoch 88/500 Train Src Label Loss: 0.621 Train Src Domain Loss: 0.607 Train Tgt Domain Loss: 0.638
Epoch 88/500 Val Src Label Loss: 1.784 Val Src Domain Loss: 0.691 Val Tgt Domain Loss: 0.551 Val Score 2.249
Epoch 89/500 Train Src Label Loss: 0.616 Train Src Domain Loss: 0.607 Train Tgt Domain Loss: 0.636
Epoch 89/500 Val Src Label Loss: 1.804 Val Src Domain Loss: 0.697 Val Tgt Domain Loss: 0.544 Val Score 2.253
Epoch 90/500 Train Src Label Loss: 0.621 Train Src Domain Loss: 0.615 Train Tgt Domain Loss: 0.63
Epoch 90/500 Val Src Label Loss: 1.785 Val Src Domain Loss: 0.703 Val Tgt Domain Loss: 0.54 Val Score 2.263
Epoch 91/500 Train Src Label Loss: 0.625 Train Src Domain Loss: 0.622 Train Tgt Domain Loss: 0.622
Epoch 91/500 Val Src Label Loss: 1.796 Val Src Domain Loss: 0.703 Val Tgt Domain Loss: 0.538 Val Score 2.266
Epoch 92/500 Train Src Label Loss: 0.613 Train Src Domain Loss: 0.621 Train Tgt Domain Loss: 0.623
Epoch 92/500 Val Src Label Loss: 1.791 Val Src Domain Loss: 0.701 Val T

Epoch 128/500 Train Src Label Loss: 0.592 Train Src Domain Loss: 0.609 Train Tgt Domain Loss: 0.619
Epoch 128/500 Val Src Label Loss: 1.827 Val Src Domain Loss: 0.689 Val Tgt Domain Loss: 0.541 Val Score 2.28
Epoch 129/500 Train Src Label Loss: 0.603 Train Src Domain Loss: 0.6 Train Tgt Domain Loss: 0.628
Epoch 129/500 Val Src Label Loss: 1.829 Val Src Domain Loss: 0.692 Val Tgt Domain Loss: 0.539 Val Score 2.29
Epoch 130/500 Train Src Label Loss: 0.6 Train Src Domain Loss: 0.603 Train Tgt Domain Loss: 0.625
Epoch 130/500 Val Src Label Loss: 1.811 Val Src Domain Loss: 0.704 Val Tgt Domain Loss: 0.532 Val Score 2.287
Epoch 131/500 Train Src Label Loss: 0.595 Train Src Domain Loss: 0.611 Train Tgt Domain Loss: 0.621
Epoch 131/500 Val Src Label Loss: 1.822 Val Src Domain Loss: 0.702 Val Tgt Domain Loss: 0.533 Val Score 2.281
Epoch 132/500 Train Src Label Loss: 0.595 Train Src Domain Loss: 0.611 Train Tgt Domain Loss: 0.617
Epoch 132/500 Val Src Label Loss: 1.832 Val Src Domain Loss: 0.69 

Epoch 169/500 Train Src Label Loss: 0.576 Train Src Domain Loss: 0.59 Train Tgt Domain Loss: 0.623
Epoch 169/500 Val Src Label Loss: 1.863 Val Src Domain Loss: 0.703 Val Tgt Domain Loss: 0.529 Val Score 2.299
Epoch 170/500 Train Src Label Loss: 0.579 Train Src Domain Loss: 0.602 Train Tgt Domain Loss: 0.611
Epoch 170/500 Val Src Label Loss: 1.885 Val Src Domain Loss: 0.707 Val Tgt Domain Loss: 0.52 Val Score 2.302
Epoch 171/500 Train Src Label Loss: 0.576 Train Src Domain Loss: 0.607 Train Tgt Domain Loss: 0.605
Epoch 171/500 Val Src Label Loss: 1.849 Val Src Domain Loss: 0.707 Val Tgt Domain Loss: 0.523 Val Score 2.304
Epoch 172/500 Train Src Label Loss: 0.566 Train Src Domain Loss: 0.607 Train Tgt Domain Loss: 0.606
Epoch 172/500 Val Src Label Loss: 1.863 Val Src Domain Loss: 0.701 Val Tgt Domain Loss: 0.526 Val Score 2.308
Epoch 173/500 Train Src Label Loss: 0.566 Train Src Domain Loss: 0.601 Train Tgt Domain Loss: 0.612
Epoch 173/500 Val Src Label Loss: 1.859 Val Src Domain Loss: 0

...

Epoch 290/500 Train Src Label Loss: 0.508 Train Src Domain Loss: 0.578 Train Tgt Domain Loss: 0.602
Epoch 290/500 Val Src Label Loss: 1.967 Val Src Domain Loss: 0.7 Val Tgt Domain Loss: 0.526 Val Score 2.352
Epoch 291/500 Train Src Label Loss: 0.513 Train Src Domain Loss: 0.573 Train Tgt Domain Loss: 0.608
Epoch 291/500 Val Src Label Loss: 1.975 Val Src Domain Loss: 0.701 Val Tgt Domain Loss: 0.525 Val Score 2.356
Epoch 292/500 Train Src Label Loss: 0.506 Train Src Domain Loss: 0.574 Train Tgt Domain Loss: 0.606
Epoch 292/500 Val Src Label Loss: 1.993 Val Src Domain Loss: 0.705 Val Tgt Domain Loss: 0.523 Val Score 2.36
Epoch 293/500 Train Src Label Loss: 0.513 Train Src Domain Loss: 0.576 Train Tgt Domain Loss: 0.601
Epoch 293/500 Val Src Label Loss: 2.001 Val Src Domain Loss: 0.708 Val Tgt Domain Loss: 0.519 Val Score 2.363
Epoch 294/500 Train Src Label Loss: 0.503 Train Src Domain Loss: 0.581 Train Tgt Domain Loss: 0.6
Epoch 294/500 Val Src Label Loss: 2.009 Val Src Domain Loss: 0.71

Epoch 333/500 Train Src Label Loss: 0.485 Train Src Domain Loss: 0.593 Train Tgt Domain Loss: 0.58
Epoch 333/500 Val Src Label Loss: 2.025 Val Src Domain Loss: 0.707 Val Tgt Domain Loss: 0.517 Val Score 2.384
Epoch 334/500 Train Src Label Loss: 0.487 Train Src Domain Loss: 0.576 Train Tgt Domain Loss: 0.596
Epoch 334/500 Val Src Label Loss: 2.025 Val Src Domain Loss: 0.692 Val Tgt Domain Loss: 0.522 Val Score 2.385
Epoch 335/500 Train Src Label Loss: 0.488 Train Src Domain Loss: 0.568 Train Tgt Domain Loss: 0.605
Epoch 335/500 Val Src Label Loss: 2.004 Val Src Domain Loss: 0.699 Val Tgt Domain Loss: 0.521 Val Score 2.375
Epoch 336/500 Train Src Label Loss: 0.492 Train Src Domain Loss: 0.571 Train Tgt Domain Loss: 0.6
Epoch 336/500 Val Src Label Loss: 2.041 Val Src Domain Loss: 0.71 Val Tgt Domain Loss: 0.517 Val Score 2.371
Epoch 337/500 Train Src Label Loss: 0.484 Train Src Domain Loss: 0.576 Train Tgt Domain Loss: 0.592
Epoch 337/500 Val Src Label Loss: 2.036 Val Src Domain Loss: 0.7

Epoch 374/500 Train Src Label Loss: 0.469 Train Src Domain Loss: 0.589 Train Tgt Domain Loss: 0.577
Epoch 374/500 Val Src Label Loss: 2.087 Val Src Domain Loss: 0.708 Val Tgt Domain Loss: 0.514 Val Score 2.392
Epoch 375/500 Train Src Label Loss: 0.472 Train Src Domain Loss: 0.576 Train Tgt Domain Loss: 0.588
Epoch 375/500 Val Src Label Loss: 2.082 Val Src Domain Loss: 0.687 Val Tgt Domain Loss: 0.53 Val Score 2.395
Epoch 376/500 Train Src Label Loss: 0.464 Train Src Domain Loss: 0.555 Train Tgt Domain Loss: 0.612
Epoch 376/500 Val Src Label Loss: 2.099 Val Src Domain Loss: 0.678 Val Tgt Domain Loss: 0.537 Val Score 2.401
Epoch 377/500 Train Src Label Loss: 0.464 Train Src Domain Loss: 0.542 Train Tgt Domain Loss: 0.62
Epoch 377/500 Val Src Label Loss: 2.111 Val Src Domain Loss: 0.708 Val Tgt Domain Loss: 0.515 Val Score 2.407
Epoch 378/500 Train Src Label Loss: 0.47 Train Src Domain Loss: 0.57 Train Tgt Domain Loss: 0.592
Epoch 378/500 Val Src Label Loss: 2.071 Val Src Domain Loss: 0.7

Epoch 414/500 Train Src Label Loss: 0.442 Train Src Domain Loss: 0.585 Train Tgt Domain Loss: 0.57
Epoch 414/500 Val Src Label Loss: 2.132 Val Src Domain Loss: 0.732 Val Tgt Domain Loss: 0.502 Val Score 2.414
Epoch 415/500 Train Src Label Loss: 0.448 Train Src Domain Loss: 0.588 Train Tgt Domain Loss: 0.567
Epoch 415/500 Val Src Label Loss: 2.148 Val Src Domain Loss: 0.705 Val Tgt Domain Loss: 0.518 Val Score 2.415
Epoch 416/500 Train Src Label Loss: 0.438 Train Src Domain Loss: 0.569 Train Tgt Domain Loss: 0.59
Epoch 416/500 Val Src Label Loss: 2.136 Val Src Domain Loss: 0.675 Val Tgt Domain Loss: 0.537 Val Score 2.418
Epoch 417/500 Train Src Label Loss: 0.448 Train Src Domain Loss: 0.546 Train Tgt Domain Loss: 0.612
Epoch 417/500 Val Src Label Loss: 2.152 Val Src Domain Loss: 0.679 Val Tgt Domain Loss: 0.534 Val Score 2.418
Epoch 418/500 Train Src Label Loss: 0.441 Train Src Domain Loss: 0.547 Train Tgt Domain Loss: 0.611
Epoch 418/500 Val Src Label Loss: 2.152 Val Src Domain Loss: 0

Epoch 457/500 Train Src Label Loss: 0.429 Train Src Domain Loss: 0.558 Train Tgt Domain Loss: 0.593
Epoch 457/500 Val Src Label Loss: 2.18 Val Src Domain Loss: 0.733 Val Tgt Domain Loss: 0.499 Val Score 2.44
Epoch 458/500 Train Src Label Loss: 0.428 Train Src Domain Loss: 0.58 Train Tgt Domain Loss: 0.572
Epoch 458/500 Val Src Label Loss: 2.175 Val Src Domain Loss: 0.725 Val Tgt Domain Loss: 0.506 Val Score 2.428
Epoch 459/500 Train Src Label Loss: 0.425 Train Src Domain Loss: 0.579 Train Tgt Domain Loss: 0.571
Epoch 459/500 Val Src Label Loss: 2.189 Val Src Domain Loss: 0.689 Val Tgt Domain Loss: 0.522 Val Score 2.427
Epoch 460/500 Train Src Label Loss: 0.424 Train Src Domain Loss: 0.551 Train Tgt Domain Loss: 0.595
Epoch 460/500 Val Src Label Loss: 2.179 Val Src Domain Loss: 0.686 Val Tgt Domain Loss: 0.533 Val Score 2.433
Epoch 461/500 Train Src Label Loss: 0.427 Train Src Domain Loss: 0.542 Train Tgt Domain Loss: 0.608
Epoch 461/500 Val Src Label Loss: 2.18 Val Src Domain Loss: 0.6

In [27]:
val_loss_src_label, val_loss_src_domain, val_loss_tgt_domain

(2.2199935913085938, 0.7127935886383057, 0.514947772026062)

## 2017-2018 ##

* Test:
** 2017-2018 (then adapt 2016-2017 adapt-val 2018)

In [28]:
data_2017_2019 = temp_partition_arxiv(data, year_bound=[-1,2017,2019], proportion=1.0)
data_2017_2019 = data_2017_2019.to(device)

In [29]:
test_loss, test_acc = test(encoder_2014_2016_2017, mlp_2014_2016_2017, data_2017_2019, evaluator)
test_acc_list.append(test_acc)

In [30]:
print(f"Test Loss: {round(test_loss,3)} Test Acc: {round(test_acc,3)}")

Test Loss: 2.565 Test Acc: 0.508


In [31]:
encoder_2016_2018_2019, mlp_2016_2018_2019, val_loss_src_label, val_loss_src_domain, val_loss_tgt_domain = continual_adapt(data_2012_2013, [2016,2018,2019], encoder_2014_2016_2017, mlp_2014_2016_2017, device, lambda_coeff=lambda_coeff, lr=lr)

Start partitioning data...
Finish partitioning data...
Epoch 1/500 Train Src Label Loss: 0.406 Train Src Domain Loss: 0.863 Train Tgt Domain Loss: 0.62
Epoch 1/500 Val Src Label Loss: 2.301 Val Src Domain Loss: 0.845 Val Tgt Domain Loss: 0.508 Val Score 2.317
Epoch 2/500 Train Src Label Loss: 0.477 Train Src Domain Loss: 0.81 Train Tgt Domain Loss: 0.543
Epoch 2/500 Val Src Label Loss: 2.236 Val Src Domain Loss: 0.824 Val Tgt Domain Loss: 0.445 Val Score 2.391
Epoch 3/500 Train Src Label Loss: 0.419 Train Src Domain Loss: 0.758 Train Tgt Domain Loss: 0.514
Epoch 3/500 Val Src Label Loss: 2.24 Val Src Domain Loss: 0.804 Val Tgt Domain Loss: 0.419 Val Score 2.432
Epoch 4/500 Train Src Label Loss: 0.439 Train Src Domain Loss: 0.719 Train Tgt Domain Loss: 0.515
Epoch 4/500 Val Src Label Loss: 2.259 Val Src Domain Loss: 0.782 Val Tgt Domain Loss: 0.421 Val Score 2.438
Epoch 5/500 Train Src Label Loss: 0.441 Train Src Domain Loss: 0.678 Train Tgt Domain Loss: 0.538
Epoch 5/500 Val Src Label 

Epoch 41/500 Train Src Label Loss: 0.397 Train Src Domain Loss: 0.528 Train Tgt Domain Loss: 0.58
Epoch 41/500 Val Src Label Loss: 2.251 Val Src Domain Loss: 0.603 Val Tgt Domain Loss: 0.456 Val Score 2.411
Epoch 42/500 Train Src Label Loss: 0.4 Train Src Domain Loss: 0.521 Train Tgt Domain Loss: 0.586
Epoch 42/500 Val Src Label Loss: 2.263 Val Src Domain Loss: 0.605 Val Tgt Domain Loss: 0.458 Val Score 2.411
Epoch 43/500 Train Src Label Loss: 0.398 Train Src Domain Loss: 0.517 Train Tgt Domain Loss: 0.588
Epoch 43/500 Val Src Label Loss: 2.256 Val Src Domain Loss: 0.609 Val Tgt Domain Loss: 0.454 Val Score 2.41
Epoch 44/500 Train Src Label Loss: 0.394 Train Src Domain Loss: 0.524 Train Tgt Domain Loss: 0.579
Epoch 44/500 Val Src Label Loss: 2.273 Val Src Domain Loss: 0.619 Val Tgt Domain Loss: 0.444 Val Score 2.404
Epoch 45/500 Train Src Label Loss: 0.398 Train Src Domain Loss: 0.529 Train Tgt Domain Loss: 0.572
Epoch 45/500 Val Src Label Loss: 2.239 Val Src Domain Loss: 0.621 Val Tgt

Epoch 81/500 Train Src Label Loss: 0.39 Train Src Domain Loss: 0.524 Train Tgt Domain Loss: 0.551
Epoch 81/500 Val Src Label Loss: 2.284 Val Src Domain Loss: 0.626 Val Tgt Domain Loss: 0.417 Val Score 2.421
Epoch 82/500 Train Src Label Loss: 0.387 Train Src Domain Loss: 0.524 Train Tgt Domain Loss: 0.545
Epoch 82/500 Val Src Label Loss: 2.314 Val Src Domain Loss: 0.616 Val Tgt Domain Loss: 0.422 Val Score 2.417
Epoch 83/500 Train Src Label Loss: 0.376 Train Src Domain Loss: 0.522 Train Tgt Domain Loss: 0.549
Epoch 83/500 Val Src Label Loss: 2.309 Val Src Domain Loss: 0.608 Val Tgt Domain Loss: 0.427 Val Score 2.419
Epoch 84/500 Train Src Label Loss: 0.389 Train Src Domain Loss: 0.513 Train Tgt Domain Loss: 0.553
Epoch 84/500 Val Src Label Loss: 2.319 Val Src Domain Loss: 0.599 Val Tgt Domain Loss: 0.436 Val Score 2.422
Epoch 85/500 Train Src Label Loss: 0.384 Train Src Domain Loss: 0.506 Train Tgt Domain Loss: 0.563
Epoch 85/500 Val Src Label Loss: 2.275 Val Src Domain Loss: 0.589 Val 

Epoch 121/500 Train Src Label Loss: 0.373 Train Src Domain Loss: 0.495 Train Tgt Domain Loss: 0.551
Epoch 121/500 Val Src Label Loss: 2.309 Val Src Domain Loss: 0.611 Val Tgt Domain Loss: 0.413 Val Score 2.429
Epoch 122/500 Train Src Label Loss: 0.372 Train Src Domain Loss: 0.512 Train Tgt Domain Loss: 0.535
Epoch 122/500 Val Src Label Loss: 2.328 Val Src Domain Loss: 0.621 Val Tgt Domain Loss: 0.406 Val Score 2.427
Epoch 123/500 Train Src Label Loss: 0.378 Train Src Domain Loss: 0.517 Train Tgt Domain Loss: 0.525
Epoch 123/500 Val Src Label Loss: 2.34 Val Src Domain Loss: 0.614 Val Tgt Domain Loss: 0.41 Val Score 2.432
Epoch 124/500 Train Src Label Loss: 0.376 Train Src Domain Loss: 0.512 Train Tgt Domain Loss: 0.533
Epoch 124/500 Val Src Label Loss: 2.326 Val Src Domain Loss: 0.591 Val Tgt Domain Loss: 0.424 Val Score 2.443
Epoch 125/500 Train Src Label Loss: 0.373 Train Src Domain Loss: 0.495 Train Tgt Domain Loss: 0.55
Epoch 125/500 Val Src Label Loss: 2.357 Val Src Domain Loss: 0.

Epoch 161/500 Train Src Label Loss: 0.367 Train Src Domain Loss: 0.49 Train Tgt Domain Loss: 0.535
Epoch 161/500 Val Src Label Loss: 2.417 Val Src Domain Loss: 0.584 Val Tgt Domain Loss: 0.423 Val Score 2.449
Epoch 162/500 Train Src Label Loss: 0.358 Train Src Domain Loss: 0.48 Train Tgt Domain Loss: 0.543
Epoch 162/500 Val Src Label Loss: 2.356 Val Src Domain Loss: 0.566 Val Tgt Domain Loss: 0.426 Val Score 2.449
Epoch 163/500 Train Src Label Loss: 0.36 Train Src Domain Loss: 0.472 Train Tgt Domain Loss: 0.552
Epoch 163/500 Val Src Label Loss: 2.379 Val Src Domain Loss: 0.57 Val Tgt Domain Loss: 0.43 Val Score 2.45
Epoch 164/500 Train Src Label Loss: 0.358 Train Src Domain Loss: 0.472 Train Tgt Domain Loss: 0.552
Epoch 164/500 Val Src Label Loss: 2.398 Val Src Domain Loss: 0.578 Val Tgt Domain Loss: 0.426 Val Score 2.453
Epoch 165/500 Train Src Label Loss: 0.358 Train Src Domain Loss: 0.479 Train Tgt Domain Loss: 0.546
Epoch 165/500 Val Src Label Loss: 2.428 Val Src Domain Loss: 0.59 

...

Epoch 323/500 Train Src Label Loss: 0.314 Train Src Domain Loss: 0.456 Train Tgt Domain Loss: 0.508
Epoch 323/500 Val Src Label Loss: 2.562 Val Src Domain Loss: 0.571 Val Tgt Domain Loss: 0.407 Val Score 2.495
Epoch 324/500 Train Src Label Loss: 0.313 Train Src Domain Loss: 0.455 Train Tgt Domain Loss: 0.513
Epoch 324/500 Val Src Label Loss: 2.566 Val Src Domain Loss: 0.564 Val Tgt Domain Loss: 0.413 Val Score 2.493
Epoch 325/500 Train Src Label Loss: 0.316 Train Src Domain Loss: 0.445 Train Tgt Domain Loss: 0.518
Epoch 325/500 Val Src Label Loss: 2.581 Val Src Domain Loss: 0.546 Val Tgt Domain Loss: 0.418 Val Score 2.491
Epoch 326/500 Train Src Label Loss: 0.313 Train Src Domain Loss: 0.435 Train Tgt Domain Loss: 0.529
Epoch 326/500 Val Src Label Loss: 2.523 Val Src Domain Loss: 0.549 Val Tgt Domain Loss: 0.411 Val Score 2.484
Epoch 327/500 Train Src Label Loss: 0.315 Train Src Domain Loss: 0.437 Train Tgt Domain Loss: 0.527
Epoch 327/500 Val Src Label Loss: 2.583 Val Src Domain Loss:

Epoch 365/500 Train Src Label Loss: 0.301 Train Src Domain Loss: 0.456 Train Tgt Domain Loss: 0.502
Epoch 365/500 Val Src Label Loss: 2.591 Val Src Domain Loss: 0.601 Val Tgt Domain Loss: 0.384 Val Score 2.512
Epoch 366/500 Train Src Label Loss: 0.304 Train Src Domain Loss: 0.468 Train Tgt Domain Loss: 0.488
Epoch 366/500 Val Src Label Loss: 2.615 Val Src Domain Loss: 0.594 Val Tgt Domain Loss: 0.384 Val Score 2.503
Epoch 367/500 Train Src Label Loss: 0.309 Train Src Domain Loss: 0.469 Train Tgt Domain Loss: 0.487
Epoch 367/500 Val Src Label Loss: 2.655 Val Src Domain Loss: 0.573 Val Tgt Domain Loss: 0.398 Val Score 2.496
Epoch 368/500 Train Src Label Loss: 0.296 Train Src Domain Loss: 0.451 Train Tgt Domain Loss: 0.502
Epoch 368/500 Val Src Label Loss: 2.629 Val Src Domain Loss: 0.562 Val Tgt Domain Loss: 0.406 Val Score 2.497
Epoch 369/500 Train Src Label Loss: 0.306 Train Src Domain Loss: 0.439 Train Tgt Domain Loss: 0.514
Epoch 369/500 Val Src Label Loss: 2.654 Val Src Domain Loss:

Epoch 406/500 Train Src Label Loss: 0.3 Train Src Domain Loss: 0.448 Train Tgt Domain Loss: 0.498
Epoch 406/500 Val Src Label Loss: 2.688 Val Src Domain Loss: 0.571 Val Tgt Domain Loss: 0.397 Val Score 2.511
Epoch 407/500 Train Src Label Loss: 0.296 Train Src Domain Loss: 0.444 Train Tgt Domain Loss: 0.499
Epoch 407/500 Val Src Label Loss: 2.676 Val Src Domain Loss: 0.573 Val Tgt Domain Loss: 0.394 Val Score 2.507
Epoch 408/500 Train Src Label Loss: 0.299 Train Src Domain Loss: 0.449 Train Tgt Domain Loss: 0.499
Epoch 408/500 Val Src Label Loss: 2.67 Val Src Domain Loss: 0.574 Val Tgt Domain Loss: 0.4 Val Score 2.513
Epoch 409/500 Train Src Label Loss: 0.294 Train Src Domain Loss: 0.445 Train Tgt Domain Loss: 0.505
Epoch 409/500 Val Src Label Loss: 2.679 Val Src Domain Loss: 0.567 Val Tgt Domain Loss: 0.4 Val Score 2.517
Epoch 410/500 Train Src Label Loss: 0.293 Train Src Domain Loss: 0.437 Train Tgt Domain Loss: 0.505
Epoch 410/500 Val Src Label Loss: 2.63 Val Src Domain Loss: 0.566 V

Epoch 446/500 Train Src Label Loss: 0.288 Train Src Domain Loss: 0.437 Train Tgt Domain Loss: 0.505
Epoch 446/500 Val Src Label Loss: 2.669 Val Src Domain Loss: 0.569 Val Tgt Domain Loss: 0.399 Val Score 2.516
Epoch 447/500 Train Src Label Loss: 0.288 Train Src Domain Loss: 0.44 Train Tgt Domain Loss: 0.501
Epoch 447/500 Val Src Label Loss: 2.69 Val Src Domain Loss: 0.583 Val Tgt Domain Loss: 0.396 Val Score 2.515
Epoch 448/500 Train Src Label Loss: 0.28 Train Src Domain Loss: 0.444 Train Tgt Domain Loss: 0.497
Epoch 448/500 Val Src Label Loss: 2.699 Val Src Domain Loss: 0.591 Val Tgt Domain Loss: 0.391 Val Score 2.517
Epoch 449/500 Train Src Label Loss: 0.288 Train Src Domain Loss: 0.445 Train Tgt Domain Loss: 0.493
Epoch 449/500 Val Src Label Loss: 2.682 Val Src Domain Loss: 0.589 Val Tgt Domain Loss: 0.391 Val Score 2.522
Epoch 450/500 Train Src Label Loss: 0.288 Train Src Domain Loss: 0.451 Train Tgt Domain Loss: 0.491
Epoch 450/500 Val Src Label Loss: 2.638 Val Src Domain Loss: 0.

Epoch 486/500 Train Src Label Loss: 0.284 Train Src Domain Loss: 0.429 Train Tgt Domain Loss: 0.502
Epoch 486/500 Val Src Label Loss: 2.745 Val Src Domain Loss: 0.584 Val Tgt Domain Loss: 0.391 Val Score 2.532
Epoch 487/500 Train Src Label Loss: 0.277 Train Src Domain Loss: 0.448 Train Tgt Domain Loss: 0.488
Epoch 487/500 Val Src Label Loss: 2.728 Val Src Domain Loss: 0.593 Val Tgt Domain Loss: 0.386 Val Score 2.53
Epoch 488/500 Train Src Label Loss: 0.27 Train Src Domain Loss: 0.45 Train Tgt Domain Loss: 0.485
Epoch 488/500 Val Src Label Loss: 2.778 Val Src Domain Loss: 0.586 Val Tgt Domain Loss: 0.388 Val Score 2.532
Epoch 489/500 Train Src Label Loss: 0.273 Train Src Domain Loss: 0.452 Train Tgt Domain Loss: 0.487
Epoch 489/500 Val Src Label Loss: 2.771 Val Src Domain Loss: 0.575 Val Tgt Domain Loss: 0.392 Val Score 2.535
Epoch 490/500 Train Src Label Loss: 0.265 Train Src Domain Loss: 0.44 Train Tgt Domain Loss: 0.492
Epoch 490/500 Val Src Label Loss: 2.747 Val Src Domain Loss: 0.5

In [32]:
val_loss_src_label, val_loss_src_domain, val_loss_tgt_domain

(2.7472541332244873, 0.5741910934448242, 0.39690327644348145)

## 2019-2020 ##

In [33]:
data_2019_2021 = temp_partition_arxiv(data, year_bound=[-1,2019,2021], proportion=1.0)
data_2019_2021 = data_2019_2021.to(device)

In [34]:
test_loss, test_acc = test(encoder_2016_2018_2019, mlp_2016_2018_2019, data_2019_2021, evaluator)
test_acc_list.append(test_acc)

In [35]:
print(f"Test Loss: {round(test_loss,3)} Test Acc: {round(test_acc,3)}")

Test Loss: 3.079 Test Acc: 0.504


In [36]:
print(test_acc_list)

[0.5907802649083232, 0.5425037005709452, 0.5080892254249527, 0.504454457543773]


In [37]:
print(sum(test_acc_list) / len(test_acc_list))

0.5364569121119986
